## Vivacomet
**Vivacomet Notebook, is the combination of diffusion notebook and spatial dFBA notebook.** 
It simulates the environment and growth of organisms within it. The environment is represented as a grid, where each bin contains specific amounts of various molecules that organisms in that bin can utilize. The simulation takes into account the diffusion and advection of molecules, as well as the biomass of species. Additionally, Vivacomet simulates the growth of organisms in each bin, which is influenced by the availability of molecules in that bin.

In [3]:
#imports
import numpy as np
from processes.diffusion_field import DiffusionField
from processes.spatial_dfba import SpatialDFBA
from plots.field import plot_objective_flux, plot_fields_temporal
from vivarium.core.engine import Engine
import base64
import shutil
import matplotlib.pyplot as plt
from IPython.display import display, HTML
import imageio.v2 as imageio
import os
import io
import warnings

In [4]:
#config
comets_config = {
    'total_time':200 ,
    'bounds': [10, 10],
    'nbins': [5, 5],
}

# Parameters shared by both processes
shared_params = {
    'bounds': comets_config['bounds'],
    'nbins': comets_config['nbins'],
    'molecules': ['glucose', 'oxygen'],
}

species_info = [
    {
        "name": "Alteromonas",
        "model": '../data/Alteromonas_Model.xml',
        "flux_id_map": {
            "glucose": "EX_cpd00027_e0",
            "oxygen": "EX_cpd00007_e0"
        },
        "kinetic_params": {
            "glucose": (0.5, 0.0005),
            "oxygen": (0.3, 0.0005),
        },
        "fixed_bounds": {
            'EX_cpd00149_e0': (-100, 1000)
        }
    },
    {
        "name": "ecoli",
        "model": '../data/iECW_1372.xml',
        "flux_id_map": {
            "glucose": "EX_glc__D_e",
            "oxygen": "EX_o2_e"
        },
        "kinetic_params": {
            "glucose": (0.4, 0.6),
            "oxygen": (0.25, 0.6),
        },
        "fixed_bounds": {
            'EX_fe3dhbzs_e': (-1000, 1000)
        }
    }
]

# Specific parameters for Diffusion Field
diffusion_field_params = {
    **shared_params,
    'default_diffusion_dt': 0.001,
    'default_diffusion_rate': 2E-5,
    'diffusion': {
        'glucose': 2.0E-2,
        'oxygen':2.7E-1,
        'Alteromonas': 1.0E-2, # diffusion rate for Alteromonas
        'ecoli': 1.0E-2 
    },
    'advection': {
        'glucose': (0.01, 0.02),
        'oxygen': (0.02, 0.01),
        'Alteromonas': (0.01, 0.01), # advection vector for Alteromonas
        'ecoli': (0.01, 0.01)
    },
    'clamp_edges': {
        'glucose': 0.5, 
        'oxygen': 0.5,
    }
}

# Specific parameters for Spatial DFBA
spatial_dfba_params = {
    **shared_params,
    'species_info': species_info
}

initial_state_config = {
    'random': {
        'glucose': 200.0,  # Max random value for glucose
        'oxygen': 200.0,   # Max random value for oxygen
        'species': {
            'ecoli': 0.5,   # Max random value for E. coli biomass
            'Alteromonas': 0.5   # Max random value for Alteromonas biomass
        }
    }
}


In [5]:
# create the two processes
diffusion_field = DiffusionField(parameters=diffusion_field_params)
spatial_dfba = SpatialDFBA(parameters=spatial_dfba_params)

# set the initial state for diffusion field
initial_state_diffusion_field = diffusion_field.initial_state({'random': 1.0})

# set the initial state for spatial dfba
initial_state_spatial_dfba = spatial_dfba.initial_state(initial_state_config)

# Merge the initial states
initial_state = {
    'fields': initial_state_diffusion_field['fields'],
    'species': initial_state_spatial_dfba['species'],
}

# make the composite simulation and run it
sim = Engine(
    processes={
        'diffusion_process': diffusion_field,
        'fba_process': spatial_dfba
    },
    topology={
        'diffusion_process': {
            'fields': ('fields',),
            'species': ('species',),
            'dimensions': ('dimensions',),
        },
        'fba_process': {
            'fields': ('fields',),
            'species': ('species',),
            'exchange_fluxes': ('exchange_fluxes',),
            'dimensions': ('dimensions',),
        }
    },
    initial_state=initial_state
)
sim.update(comets_config['total_time'])

# retrieve the results and plot them
data = sim.emitter.get_timeseries()
desired_time_points = [1, 3, comets_config['total_time'] - 1]

ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Loaded model for Alteromonas
Loaded model for ecoli

Simulation ID: c5c2c1e6-2ce8-11ef-9cad-ba21c95c07d9
Created: 06/17/2024 at 16:32:53
Completed in 561.53 seconds


## plot
This grid plot visualizes the temporal changes of biomass (E. coli, Alteromonas, and total) in the top row, while the bottom row depicts the evolving concentrations of glucose and oxygen molecules across the grid, influenced by diffusion, advection, and sinking.

In [ ]:
total_time=comets_config["total_time"]
def precompute_global_min_max(data, names, data_key):
    num_elements = len(names)
    global_min = [np.inf] * num_elements
    global_max = [-np.inf] * num_elements

    for time in data["time"]:
        time_index = int(data["time"].index(time))  # Ensure time_index is an integer
        for j, element_id in enumerate(names):
            current_data = data[data_key][element_id][time_index]
            global_min[j] = min(global_min[j], np.min(current_data))
            global_max[j] = max(global_max[j], np.max(current_data))
    
    return global_min, global_max

def plot_elements_to_gif(data, total_time, element_names, data_key, temp_dir, file_prefix, include_total_biomass=False):
    valid_time_points = list(range(total_time + 1))
    num_elements = len(element_names)
    images = []

    global_min, global_max = precompute_global_min_max(data, element_names, data_key)
    
    if include_total_biomass:
        global_min.append(np.inf)
        global_max.append(-np.inf)
        for time in valid_time_points:
            time_index = int(data["time"].index(time))  # Ensure time_index is an integer
            total_biomass = np.zeros_like(data[data_key][element_names[0]][time_index])
            for element_id in element_names:
                total_biomass += data[data_key][element_id][time_index]
            global_min[-1] = min(global_min[-1], np.min(total_biomass))
            global_max[-1] = max(global_max[-1], np.max(total_biomass))

    for time in valid_time_points:
        time_index = int(data["time"].index(time))  # Ensure time_index is an integer
        if include_total_biomass:
            total_biomass = np.zeros_like(data[data_key][element_names[0]][time_index])
        
        fig, axs = plt.subplots(1, num_elements + (1 if include_total_biomass else 0), figsize=(num_elements * 5, 5), squeeze=False)

        for j, element_id in enumerate(element_names):
            current_data = data[data_key][element_id][time_index]
            if include_total_biomass:
                total_biomass += current_data
            im = axs[0, j].imshow(current_data, cmap='viridis', vmin=global_min[j], vmax=global_max[j])
            axs[0, j].set_title(element_id)
            axs[0, j].set_xticks([])
            axs[0, j].set_yticks([])
            plt.colorbar(im, ax=axs[0, j], fraction=0.046, pad=0.04)

        if include_total_biomass:
            im = axs[0, -1].imshow(total_biomass, cmap='viridis', vmin=global_min[-1], vmax=global_max[-1])
            axs[0, -1].set_title("Total Biomass")
            axs[0, -1].set_xticks([])
            axs[0, -1].set_yticks([])
            plt.colorbar(im, ax=axs[0, -1], fraction=0.046, pad=0.04)

        plt.suptitle(f'Time: {time}', fontsize=16)
        plt.tight_layout()
        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=120)
        buf.seek(0)
        images.append(imageio.imread(buf))
        buf.close()
        plt.close(fig)

    for i, img in enumerate(images):
        imageio.imwrite(os.path.join(temp_dir, f'{file_prefix}_{i}.png'), img)

def combine_gifs(output_filename, temp_dir, num_images):
    combined_images = []
    for i in range(num_images):
        obj_flux_img = imageio.imread(os.path.join(temp_dir, f'obj_flux_{i}.png'))
        molecule_img = imageio.imread(os.path.join(temp_dir, f'molecule_{i}.png'))
        
        combined_img = np.vstack((obj_flux_img, molecule_img))
        combined_images.append(combined_img)

    imageio.mimsave(output_filename, combined_images, duration=0.5, loop=0)

    with open(output_filename, 'rb') as file:
        data = file.read()
        data_url = 'data:image/gif;base64,' + base64.b64encode(data).decode()
    display(HTML(f'<img src="{data_url}" alt="Combined GIF" style="max-width:100%;"/>'))

temp_dir = 'temp_images'
os.makedirs(temp_dir, exist_ok=True)

plot_elements_to_gif(data, total_time=total_time, element_names=['ecoli', 'Alteromonas'], data_key='species', temp_dir=temp_dir, file_prefix='obj_flux', include_total_biomass=True)
plot_elements_to_gif(data, total_time=total_time, element_names=['glucose', 'oxygen'], data_key='fields', temp_dir=temp_dir, file_prefix='molecule')

combine_gifs('combined_over_time.gif', temp_dir, 201)

shutil.rmtree(temp_dir)